In [28]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [29]:
model_version = 4
fng_feature_group_version = 1
solana_feature_group_version = 1
bitcoin_feature_group_version = 1

In [30]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1736020046

In [31]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-04 21:47:27,077 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-04 21:47:27,085 INFO: Initializing external client
2025-01-04 21:47:27,085 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 21:47:28,893 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164448


In [32]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732564046.0

In [33]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=solana_feature_group_version
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.73s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732579200,233.96,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889
1,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184
2,1732752000,242.26,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987
3,1732838400,237.59,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213
4,1732924800,243.47,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804


In [34]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=bitcoin_feature_group_version
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.63s) 


,timestamp,open
0,1732579200,93000.0
1,1732665600,91914.5
2,1732752000,95952.9
3,1732838400,95637.8
4,1732924800,97478.7


In [35]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=fng_feature_group_version
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['fng_classification'] = fng_batch_data['fng_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.63s) 


,fng_value,fng_classification,timestamp
0,79,4,1.732579e+09
1,75,3,1.732666e+09
2,77,4,1.732752e+09
3,78,4,1.732838e+09
4,84,4,1.732925e+09


In [36]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=model_version,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/vc/103l9n915p1_fcsd6kw28dsm0000gn/T/f7027e41-03d2-4efb-a907-4aeb7a694d7c/solana_price_daily_model_10d/4


<Sequential name=sequential_7, built=True>

In [37]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,fng_value,fng_classification,open_solana
0,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889,93000.0,79,4,233.96
1,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68
2,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987,95952.9,77,4,242.26
3,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213,95637.8,78,4,237.59
4,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804,97478.7,84,4,243.47


In [38]:
model_workspace = "solana_price_10d_scalers/" 
scaler_labels = joblib.load(model_workspace+'scaler_labels.pkl')
scaler_features = joblib.load(model_workspace+'scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [39]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'fng_value', 'fng_classification', 'open_solana'],
      dtype='object')

In [40]:
predictions = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


In [41]:
predictions

array([[0.8146669 , 0.8556554 , 0.71653044, 0.82936704, 0.78877497,
        0.7910065 , 0.72385657, 0.9152052 , 0.8212321 , 0.8136881 ]],
      dtype=float32)

In [42]:
predictions_unscaled =  scaler_labels.inverse_transform(predictions)
predictions_unscaled


array([[212.51466, 222.72243, 188.07475, 216.17557, 206.06653, 206.62227,
        189.89925, 237.5527 , 214.14964, 212.27089]], dtype=float32)

In [43]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions[0]))]

In [ ]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['timestamp']
)

In [ ]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions_unscaled[0]
result

,timestamp,open_solana
0,1736106446,212.514664
1,1736192846,222.722427
2,1736279246,188.074753
3,1736365646,216.175568
4,1736452046,206.066528
5,1736538446,206.622269
6,1736624846,189.899246
7,1736711246,237.552704
8,1736797646,214.149643
9,1736884046,212.270889


In [46]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
result= result.sort_values(by=['date'])

# result.drop(columns=["timestamp"], inplace=True)

result

,timestamp,open_solana,date
0,1736106446,212.514664,2025-01-05
1,1736192846,222.722427,2025-01-06
2,1736279246,188.074753,2025-01-07
3,1736365646,216.175568,2025-01-08
4,1736452046,206.066528,2025-01-09
5,1736538446,206.622269,2025-01-10
6,1736624846,189.899246,2025-01-11
7,1736711246,237.552704,2025-01-12
8,1736797646,214.149643,2025-01-13
9,1736884046,212.270889,2025-01-14


In [ ]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [ ]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    # overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)